In [1]:
import csv
import requests, bs4

# 放置爬取的縣市連結
URLs_area = []

# 放置爬取的資料
funeral_data = []

# 地區超連結
URLs_region = ('http://www.funeralinformation.com.tw/Detail.php?LevelNo=1568',\
              'http://www.funeralinformation.com.tw/Detail.php?LevelNo=1570',\
              'http://www.funeralinformation.com.tw/Detail.php?LevelNo=1572',\
              'http://www.funeralinformation.com.tw/Detail.php?LevelNo=1580')

In [2]:
def get_area_URLs():
    for URL_region in URLs_region:

        url = URL_region
        response = requests.get(url)
        soup = bs4.BeautifulSoup(response.text, 'lxml')

        #print(soup.prettify())

        a_tags = soup.find_all("a",class_= "link1")
#         print(a_tags)
        for index, tag in enumerate(a_tags):
            #輸出超連結的文字
            if index >= 8:
                area_url = 'http://www.funeralinformation.com.tw/Detail.php?LevelNo='+ tag.get('href')[-4:]
#                 print(area_url)
                URLs_area.append({
                "URLs":area_url
                 })
                

In [3]:
def get_every_table_data():
    i=0
    for every_URLs in URLs_area:
        #讀取連結
        area_URL = every_URLs['URLs']    
        url = area_URL
        response = requests.get(url)
        soup = bs4.BeautifulSoup(response.text, 'lxml')

        #print(soup.prettify())

        tbody_tags = soup.find_all("tbody")
        
        for index, tbody in enumerate(tbody_tags):
            td_tags = tbody.find_all("td")

            for index, td in enumerate(td_tags):
                if index >= 5:
                    #print(index,td.string)
                    if i == 0:
                        funeral_index=td.string
                        i+=1
                    elif i == 1:
                        funeral_name=td.string
                        i+=1
                    elif i == 2:
                        funeral_num=td.string
                        i+=1
                    elif i == 3:
                        funeral_addr=td.string
                        i+=1
                    elif i == 4:
                        funeral_other=td.string
                        funeral_data.append([
                        funeral_index,
                        funeral_name,
                        funeral_num,
                        funeral_addr,
                        funeral_other
                        ])
                        i = 0
                else:
                    continue
                

In [4]:
get_area_URLs()
get_every_table_data()

In [5]:
def createcsv():
    funeral_data.insert(0,['序號','設施名稱','電話','地址','備註'])
    csvfile=open('funeral_data.csv','w', newline='',encoding="utf-8")
    obj = csv.writer(csvfile)
    for row in funeral_data:
        obj.writerow(row)
    csvfile.close()                     
                

In [6]:
createcsv()